# Federated Learning

Federated learning is a machine learning paradigm that enables decentralized training of a shared model by multiple clients while preserving data privacy. The main idea behind this new paradigm is that each client trains a local model on its own data and then sends only the model updates to a central server, rather than sending the raw data. This allows the model to be trained on a large amount of data without compromising data privacy.

Federated learning was first proposed by Google in 2016 (McMahan et al., 2016) and has since been applied in various fields, such as healthcare (Hard et al., 2018), finance (Yoon et al., 2018), and natural language processing (Li et al., 2020).  For example, federated learning could be used to train a model that can make personalized recommendations for each user without requiring the raw data from each user to be shared with a central server. This is the mechanism by which the model is trained on data, while adhering to data privacy requirements.

Federated learning is a machine learning approach that offers numerous advantages over traditional centralized methods. Firstly, by leveraging distributed data stores, federated learning can scale to handle significantly larger datasets. Secondly, it prioritizes data privacy by avoiding the transmission of raw data to a central server. Finally, federated learning enables collaboration among multiple clients, allowing them to jointly train a shared model without compromising the security of their individual data. Overall, these benefits make federated learning a promising approach for machine learning in fields where data privacy is of the utmost importance.

Federated learning is a process in which a central server distributes a machine learning model to multiple devices. Each device trains the model on its local data and sends the updated model back to the central server. The central server then aggregates the updates from each device to improve the global model. This process is repeated until the model converges and can generate accurate predictions on new data. The key concepts within this process are:


* Client: refers to a device or edge node that holds a local dataset and actively participates in the training of the federated model.
* Server: represents the central entity that coordinates the training of the federated model and receives model updates from the clients to aggregate into a new version of the global model.
* Federated dataset: the collection of decentralized datasets from different clients that are used to train the federated model through collaborative learning.
* Federated model: a machine learning model that is trained on the federated dataset using federated learning to make accurate predictions on new data while preserving the privacy of each client's data.
* Federated optimization: refers to the process of training the federated model using the decentralized data and model updates from the clients, which enables the model to generalize better on unseen data while preserving the privacy of the clients.
* Aggregation: the process of combining the model updates received from the clients into a new version of the global model. This can be done using various methods such as weighted averaging or other approaches.
* Rounds: refer to the number of times a federated model is distributed among clients after performing an aggregation to train the model further. The process is repeated until the model converges and achieves a satisfactory level of accuracy.


Federated learning is a relatively new approach, and as such, there are few libraries available that have adapted to it. The main actors in this space are TensorFlow Federated, PySyft, OpenMined, and Flower. Of these, TensorFlow Federated is a notable mention, although it is currently only a theoretical approach, as it does not allow for the deployment of the solution and only simulates the federated space. In contrast, Flower allows for the distribution of federated learning, although the necessary modifications can be somewhat challenging. For this tutorial, we have chosen Flower due to its more user-friendly approach and potential for future use.



# Introduction to Flower (FLWR)

Flower is a Python library that offers tools for implementing the communication and coordination aspects of federated learning. Its design emphasizes ease of use and scalability. It's important to note that Flower is not a learning framework in itself, and as such, it wraps other machine learning frameworks like TensorFlow, PyTorch, or Scikit-learn in the communication layer to enable federated learning.

To use Flower for federated learning, you will need to install the library:


When setting up a simulation environment, it's best to use the *simulation* keyword with the command to ensure the appropriate environment is loaded. On the other hand, if you plan to use Flower in a distributed setup, the command should be `!pip install flwr` on both the server and client devices. After installing the `flwr` package, you can import it into your Python code using the following statement:

In [2]:
import flwr as fl
import tensorflow as tf

FLWR provides a range of classes and functions that you can use to set up a federated learning environment, train and evaluate a model, and implement regular updates to the model. You can refer to the FLWR [documentation](https://flower.dev/docs/quickstart-tensorflow.html).  for more information. Before proceeding, it's important to note that the model you define must be serializable so that it can be sent through the network. Not all models are suitable for federated learning. For this example, we'll be using an Artificial Neural Network (ANN) based on TensorFlow, specifically Keras.

In [3]:
# Define a simple model using TensorFlow
def generate_ann():
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"],
    )
    return model

As we will be using a Deep Learning model defined in TensorFlow, it's recommended to load the data into a `Dataset` class to enable the framework to leverage any available hardware acceleration (such as a GPU on the nodes). However, due to some limitations of the framework in order to serialize the data, it has to be done manualy with the following lines of code

In [4]:
import numpy as np

NUM_CLIENTS = 5

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s


# Code to load the dataset
def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into 10 partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    return train_ds, val_ds, test_ds


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Now, let's introduce the two pieces of the puzzle: the `Client` and the `Server`. Flower starts a `Server` to coordinate the client devices and perform the orchestration of the model. The server interacts with clients through an interface called `Client`. When the server selects a particular client for training, it sends training instructions over the network. The client receives those instructions and calls one of the Client methods to run your code, which in this case involves training the neural network that we defined earlier.

Flower provides a convenient class called NumPyClient, which simplifies the implementation of the Client interface when your workload uses Keras. The NumPyClient interface defines three methods that can be implemented in the following way:

```python
#Create a class to contain the details of the client and be the interface
class MyClient(fl.client.NumPyClient):
    def __init__(self, net, train_dataset, test_dataset):
        self.model = net
        self.trainloader = train_dataset
        self.valloader = test_dataset
    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.trainloader[0],self.trainloader[1], epochs=1, batch_size=32, steps_per_epoch=3)
        return self.model.get_weights(), len(self.trainloader[0]), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.valloader[0], self.valloader[1])
        return loss, len(self.valloader[0]), {"accuracy": float(accuracy)}
```

In the preceding code, we defined the required functions for the client in this particular case. With these functions in place, we can now start a client using the following code:

```python 
# Start the client
model=generate_ann()
fl.client.start_numpy_client(server_address="[::]:8080", client=MyClient(model,trainloaders[0],valloaders[0]))
```

**Important**: In order to run the client you must need also a server running!!! You will executed both in separated terminals

The string`[::]:8080` specifies the server to which the client should connect. In this case, as the code is being run on the same machine as the server, this address is sufficient. In a truly federated workload, the only thing that needs to be changed is the `server_address` to point the client to the correct server.

Note that Jupyter usually runs on port 8080, **so you will need to use another available port if Jupyter server is running**.

**Possible exception**: if you get the following error: "failed to connect to all addresses" the you shoul use the following string to stablish the connection `localhost:8080` instead of `[::]:8080` 



The other essential piece of the puzzle is the class that will contain the server. This will be in a separate file, for example server.py, and its contents should look something like this:


```python
import flwr as fl

fl.server.start_server(config=fl.server.ServerConfig(num_rounds=3))
```

**Important**
You can use another port for running the server if you fix the following parameter in the `start_server` function: `server_address="localhost:9090"`

In this particular case, we can run two clients and a server in separate terminals of the machine. Running two client instances is as simple as executing the `python client.py` command twice in separate terminals, while the server can be started with the `python server.py` command.

Upon starting the server, we should receive an output similar to:


```shell
INFO flwr 2023-03-01 14:58:16,353 | app.py:139 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2023-03-01 14:58:16,362 | app.py:152 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flwr 2023-03-01 14:58:16,362 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-01 14:58:16,362 | server.py:270 | Requesting initial parameters from one random client
INFO flwr 2023-03-01 14:58:24,152 | server.py:274 | Received initial parameters from one random client
INFO flwr 2023-03-01 14:58:24,153 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-01 14:58:24,153 | server.py:101 | FL starting
DEBUG flwr 2023-03-01 14:58:26,118 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:27,041 | server.py:229 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-01 14:58:27,076 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-01 14:58:27,076 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:27,565 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-01 14:58:27,565 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-03-01 14:58:27,566 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:28,015 | server.py:229 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-03-01 14:58:28,027 | server.py:165 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:28,364 | server.py:179 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-03-01 14:58:28,364 | server.py:215 | fit_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:28,755 | server.py:229 | fit_round 3 received 2 results and 0 failures
DEBUG flwr 2023-03-01 14:58:28,769 | server.py:165 | evaluate_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:29,184 | server.py:179 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2023-03-01 14:58:29,185 | server.py:144 | FL finished in 5.031599427999936
INFO flwr 2023-03-01 14:58:29,185 | app.py:202 | app_fit: losses_distributed [(1, 2.3956351280212402), (2, 2.426431179046631), (3, 2.3015435934066772)]
INFO flwr 2023-03-01 14:58:29,185 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-01 14:58:29,186 | app.py:204 | app_fit: losses_centralized []

```

With that, the first federated learning approach is completed. As you can see, the system goes through three rounds of fitting and evaluating on all clients before the results are retrieved, aggregated, and redistributed to the server.

### Exercise
Implement the client and server code in two separate files. Next, execute a server and two clients from terminals. Finally, compare the results with those presented here. Were your results similar?

`Answer:`:

- **Server Output**:
```shell
INFO flwr 2023-03-02 17:54:30,400 | app.py:139 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2023-03-02 17:54:31,348 | app.py:152 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flwr 2023-03-02 17:54:31,348 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-02 17:54:31,348 | server.py:270 | Requesting initial parameters from one random client
INFO flwr 2023-03-02 17:54:39,633 | server.py:274 | Received initial parameters from one random client
INFO flwr 2023-03-02 17:54:39,634 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-02 17:54:39,634 | server.py:101 | FL starting
DEBUG flwr 2023-03-02 17:54:39,946 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-02 17:54:41,787 | server.py:229 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-02 17:54:41,801 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-02 17:54:41,801 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-02 17:54:42,080 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-02 17:54:42,081 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-03-02 17:54:42,081 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-02 17:54:42,232 | server.py:229 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-03-02 17:54:42,237 | server.py:165 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-02 17:54:42,388 | server.py:179 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-03-02 17:54:42,388 | server.py:215 | fit_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-02 17:54:42,541 | server.py:229 | fit_round 3 received 2 results and 0 failures
DEBUG flwr 2023-03-02 17:54:42,544 | server.py:165 | evaluate_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-02 17:54:42,680 | server.py:179 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2023-03-02 17:54:42,680 | server.py:144 | FL finished in 3.0458536830265075
INFO flwr 2023-03-02 17:54:42,682 | app.py:202 | app_fit: losses_distributed [(1, 2.3373066186904907), (2, 2.2974514961242676), (3, 2.298892617225647)]
INFO flwr 2023-03-02 17:54:42,682 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-02 17:54:42,682 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-02 17:54:42,682 | app.py:205 | app_fit: metrics_centralized {}
```

- **Client 1 Output**:
```shell
INFO flwr 2023-03-02 17:54:39,934 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-03-02 17:54:39,935 | connection.py:38 | ChannelConnectivity.IDLE
DEBUG flwr 2023-03-02 17:54:39,945 | connection.py:38 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-03-02 17:54:39,947 | connection.py:38 | ChannelConnectivity.READY
3/3 [==============================] - 2s 8ms/step - loss: 2.5333 - accuracy: 0.0625
16/16 [==============================] - 0s 2ms/step - loss: 2.3330 - accuracy: 0.1240
3/3 [==============================] - 0s 4ms/step - loss: 2.3757 - accuracy: 0.1250
16/16 [==============================] - 0s 2ms/step - loss: 2.3017 - accuracy: 0.1200
3/3 [==============================] - 0s 4ms/step - loss: 2.2626 - accuracy: 0.1771
16/16 [==============================] - 0s 2ms/step - loss: 2.3066 - accuracy: 0.1220
DEBUG flwr 2023-03-02 17:54:42,698 | connection.py:109 | gRPC channel closed
INFO flwr 2023-03-02 17:54:42,699 | app.py:153 | Disconnect and shut down
```
- **Client 2 Output**:
```shell
INFO flwr 2023-03-02 17:54:39,583 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-03-02 17:54:39,587 | connection.py:38 | ChannelConnectivity.IDLE
DEBUG flwr 2023-03-02 17:54:39,587 | connection.py:38 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-03-02 17:54:39,588 | connection.py:38 | ChannelConnectivity.READY
3/3 [==============================] - 2s 9ms/step - loss: 2.5118 - accuracy: 0.0625
16/16 [==============================] - 0s 2ms/step - loss: 2.3416 - accuracy: 0.0960
3/3 [==============================] - 0s 5ms/step - loss: 2.3320 - accuracy: 0.1354
16/16 [==============================] - 0s 2ms/step - loss: 2.2932 - accuracy: 0.1340
3/3 [==============================] - 0s 4ms/step - loss: 2.2790 - accuracy: 0.1250
16/16 [==============================] - 0s 2ms/step - loss: 2.2912 - accuracy: 0.1480
DEBUG flwr 2023-03-02 17:54:42,698 | connection.py:109 | gRPC channel closed
INFO flwr 2023-03-02 17:54:42,698 | app.py:153 | Disconnect and shut down
```

Regarding the clients, the information shown only gives details about the connection and disconnection to the orchestrator server, as well as the local training processes using tensorflow.

As for the orchestrator server, much more information is displayed. This will be practically identical to the example given, consisting of the following:
- Federated execution configuration (number of rounds, timeout)
- It indicates how the GRPC server is started (remote connection method), in this case without using a secure protocol (without SSL).
- The model parameters are initialized using those of a random client, indicating when they are requested and received. Note that this is the default strategy when they are not explicitly specified and the `flwr.server.strategy.FedAvg` strategy is being used.
- The following shows how the federated learning starts as such, comprising 3 rounds with training and evaluation. Note that no metrics aggregation method has been declared for the output obtained (`No fit_metrics_aggregation_fn provided` and `No evaluate_metrics_aggregation_fn provided`).
- Finally, the execution time and the error of each round (weighted averaged loss) are shown. 





# Updating parameters
The key element in this kind of approach is that the server sends the global model parameters to the client, and the client updates the local model with the parameters received from the server. It then trains the model on the local data, which changes the model parameters locally. After training, the updated model parameters are sent back to the server, or alternatively, only the gradients are sent back to the server, not the full model parameters.


In `flwr`, this communication is essentially done by two helper functions for loading and retrieving local parameters: `set_parameters` and `get_parameters`. This requirement fits well with non-state approaches such as **PyTorch** or **JAX**. As demonstrated in the previous example, `flwr` can also be used with **TensorFlow** or even **scikit-learn**.

As a result, the basic structure for any client using this library has the same format:


In [5]:
from typing import List

# Utility functions for the most common operations
def get_parameters(net) -> List[np.array]:
    return net.get_weights()


def set_parameters(net, parameters: List[np.ndarray]):
    net.set_weights(parameters)
    return net


def train(net, trainloader, epochs: int):
    net.fit(trainloader[0], trainloader[1],
            epochs=epochs, batch_size=32, steps_per_epoch=3)
    return net


def test(net, testloader):
    loss, accuracy = net.evaluate(testloader[0], testloader[1])
    return loss, accuracy

# Class to contain a Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        self.net = train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In Flower, clients can be created by extending either the `flwr.client.Client` or `flwr.client.NumPyClient` classes. In the previous example, we used `NumPyClient` because it is easier to implement and requires less code as a template. Along with the extended class, there are three main methods that need to be implemented:

* `get_parameters`: Returns the current local model parameters.
* `fit`: Receives model parameters from the server, trains the model parameters on the local data, and returns the (updated) model parameters to the server.
* `evaluate`: Receives model parameters from the server, evaluates the model parameters on the local data, and returns the evaluation result to the server.

As you can see, the `MyClient` class implemented in the previous example follows this same structure and the diference is the *id* of the client which is stored for later convinience use in accesing the data.


#### Be aware: 
Sometimes, especially when we are simulating multiple clients on a single device, it can be useful to use a function to create the client when it is required. This is particularly important in stateless frameworks, such as PyTorch, which can benefit from a more efficient implementation that creates clients only when they are required for training or evaluation. For example, the following code loads different examples for each client before discarding them:


In [6]:
def client_fn(cid) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #Take the appropiate part of the dataset
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    #Create and return the Client
    return FlowerClient(cid, net, trainloader, valloader)


Note that `myClient` cannot be used in the same sense because of the state that it keeps internally through the function `generate_ann`. However, if this state is removed, it can be used in the same way.

The clients are now set up to load, fit, and evaluate. However, we need to integrate the results from the different clients. In Flower terminology, this is known as a strategy, such as the *Federated Average (FedAvg)* strategy. In a first approach, we can use the built-in implementations of the framework, although custom strategies can also be used. Let's see an example:


In [7]:
model = generate_ann()
params = get_parameters(model)# The federated model initial parameters
del model


# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=NUM_CLIENTS,  # Never sample less than 10 clients for training
        min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
        min_available_clients=NUM_CLIENTS,  # Wait until all 10 clients are available
        initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters

)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)

2023-03-02 18:49:21.763362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO flwr 2023-03-02 18:49:22,005 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-03-02 18:49:25,846	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-02 18:49:29,414 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2093948928.0, 'object_store_memory': 1046974464.0, 'CPU': 4.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-03-02 18:49:29,420 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-02 18:49:29,427 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-03-02 18:49:29,430 | serve

(launch_and_fit pid=48061) [Client 4] fit, config: {}
(launch_and_fit pid=48060) [Client 1] fit, config: {}
(launch_and_fit pid=48059) [Client 2] fit, config: {}
(launch_and_fit pid=48062) [Client 0] fit, config: {}
3/3 [==============================] - 3s 12ms/step - loss: 2.5636 - accuracy: 0.0625
(launch_and_fit pid=48061) [Client 3] fit, config: {}


DEBUG flwr 2023-03-02 18:49:50,810 | server.py:229 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-03-02 18:49:50,899 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-02 18:49:50,901 | server.py:165 | evaluate_round 1: strategy sampled 5 clients (out of 5)


3/3 [==============================] - 1s 9ms/step - loss: 2.7900 - accuracy: 0.0729
(launch_and_evaluate pid=48061) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=48059) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=48062) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=48060) [Client 1] evaluate, config: {}
7/7 [==============================] - 2s 6ms/step - loss: 2.3147 - accuracy: 0.1100
(launch_and_evaluate pid=48061) [Client 2] loss:2.314668655395508, accuracy:0.10999999940395355
7/7 [==============================] - 2s 6ms/step - loss: 2.3395 - accuracy: 0.0850
(launch_and_evaluate pid=48062) [Client 3] loss:2.3394956588745117, accuracy:0.08500000089406967
(launch_and_evaluate pid=48059) [Client 0] loss:2.380509376525879, accuracy:0.10999999940395355
(launch_and_evaluate pid=48061) [Client 4] evaluate, config: {}
7/7 [==============================] - 1s 5ms/step - loss: 2.3441 - accuracy: 0.0950
(launch_and_evaluate pid=48060) [Client 1] loss:2.3

DEBUG flwr 2023-03-02 18:50:03,706 | server.py:179 | evaluate_round 1 received 5 results and 0 failures
WARNING flwr 2023-03-02 18:50:03,708 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-03-02 18:50:03,710 | server.py:215 | fit_round 2: strategy sampled 5 clients (out of 5)


...

(launch_and_evaluate pid=48060) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=48062) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=48059) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=48061) [Client 2] evaluate, config: {}
7/7 [==============================] - 1s 30ms/step - loss: 2.3031 - accuracy: 0.1050
(launch_and_evaluate pid=48060) [Client 0] loss:2.3030805587768555, accuracy:0.10499999672174454
7/7 [==============================] - 1s 19ms/step - loss: 2.2758 - accuracy: 0.1450
(launch_and_evaluate pid=48059) [Client 3] loss:2.2758255004882812, accuracy:0.14499999582767487
7/7 [==============================] - 1s 27ms/step - loss: 2.2884 - accuracy: 0.1500
(launch_and_evaluate pid=48060) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=48062) [Client 4] loss:2.2883729934692383, accuracy:0.15000000596046448
7/7 [==============================] - 0s 4ms/step - loss: 2.2669 - accuracy: 0.1400
(launch_and_evaluate pid=48061) [Client 2] los

DEBUG flwr 2023-03-02 18:51:29,066 | server.py:179 | evaluate_round 5 received 5 results and 0 failures
INFO flwr 2023-03-02 18:51:29,067 | server.py:144 | FL finished in 119.62401142902672
INFO flwr 2023-03-02 18:51:29,076 | app.py:202 | app_fit: losses_distributed [(1, 2.3424469947814943), (2, 2.3169223785400392), (3, 2.2887766361236572), (4, 2.3646543502807615), (5, 2.2853182315826417)]
INFO flwr 2023-03-02 18:51:29,078 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-02 18:51:29,080 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-02 18:51:29,085 | app.py:205 | app_fit: metrics_centralized {}


7/7 [==============================] - 0s 2ms/step - loss: 2.2924 - accuracy: 0.1450

History (loss, distributed):
	round 1: 2.3424469947814943
	round 2: 2.3169223785400392
	round 3: 2.2887766361236572
	round 4: 2.3646543502807615
	round 5: 2.2853182315826417


(launch_and_evaluate pid=48060) [Client 1] loss:2.2924275398254395, accuracy:0.14499999582767487


This code corresponds to the script running on the server, and it uses the simulation function to test this approach on a single device with the previously mentioned optimization to avoid overloading the device. The code generates 10 clients and randomly selects all of them (`fraction_fit = 1.0`) to train the model on all of them. After receiving the updates from the clients, the server performs the aggregation strategy before returning the global model to the clients for the next 5 rounds.

One point to highlight is that the framework is not only going to manage the `losses_distributed`, but none of the other metrics. Due to the diverse treatment of those measures, the framework cannot accurately handle the aggregation of these metrics. Users need to tell the framework how to handle and aggregate these custom metrics.

The strategy will then call these functions whenever it receives fit or evaluates metrics from clients. The two possible functions are `fit_metrics_aggregation_fn` and `evaluate_metrics_aggregation_fn`. For example, the following code creates the weighted average, and the previous example can be adapted as follows:


In [8]:
from typing import Tuple
from flwr.common import Metrics

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=5,
        min_evaluate_clients=3,
        min_available_clients=5,
        initial_parameters=fl.common.ndarrays_to_parameters(params),
        evaluate_metrics_aggregation_fn=weighted_average,  # put the metric aggregation for the evaluation
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)


INFO flwr 2023-03-02 18:51:29,344 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-03-02 18:51:38,306	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-02 18:51:39,941 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2946424014.0, 'CPU': 4.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 1473212006.0}
INFO flwr 2023-03-02 18:51:39,943 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-02 18:51:39,945 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-03-02 18:51:39,949 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-02 18:51:39,951 | server.py:101 | FL starting
DEBUG flwr 2023-03-02 18:51:39,956 | server.py:215 | fit_round 1: strategy sampled 5 clients (out of 5)
(launch_and_fit pid=48117) 2023-03-02 18:51:42.864815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is 

(launch_and_fit pid=48117) [Client 2] fit, config: {}
(launch_and_fit pid=48115) [Client 1] fit, config: {}
(launch_and_fit pid=48114) [Client 3] fit, config: {}
(launch_and_fit pid=48116) [Client 0] fit, config: {}
3/3 [==============================] - 3s 12ms/step - loss: 2.4560 - accuracy: 0.1250
(launch_and_fit pid=48114) [Client 4] fit, config: {}


DEBUG flwr 2023-03-02 18:52:00,878 | server.py:229 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-03-02 18:52:00,896 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-02 18:52:00,898 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 5)


3/3 [==============================] - 2s 4ms/step - loss: 2.4573 - accuracy: 0.0729
(launch_and_evaluate pid=48114) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=48117) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=48116) [Client 3] evaluate, config: {}
7/7 [==============================] - 1s 9ms/step - loss: 2.3295 - accuracy: 0.1000
(launch_and_evaluate pid=48114) [Client 4] loss:2.3294620513916016, accuracy:0.10000000149011612


DEBUG flwr 2023-03-02 18:52:04,102 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG flwr 2023-03-02 18:52:04,105 | server.py:215 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2023-03-02 18:52:57,957 | server.py:229 | fit_round 5 received 5 results and 0 failures
DEBUG flwr 2023-03-02 18:52:58,005 | server.py:165 | evaluate_round 5: strategy sampled 3 clients (out of 5)
(launch_and_fit pid=48115) WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_train_function.<locals>.train_function at 0x17f4c5e10> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


3/3 [==============================] - 1s 6ms/step - loss: 2.3959 - accuracy: 0.0938
(launch_and_evaluate pid=48115) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=48117) [Client 1] evaluate, config: {}
7/7 [==============================] - 0s 2ms/step - loss: 2.3194 - accuracy: 0.0950
(launch_and_evaluate pid=48117) [Client 1] loss:2.3194420337677, accuracy:0.0949999988079071
7/7 [==============================] - 1s 2ms/step - loss: 2.3064 - accuracy: 0.0850
(launch_and_evaluate pid=48115) [Client 2] loss:2.306386947631836, accuracy:0.08500000089406967
(launch_and_evaluate pid=48116) [Client 3] evaluate, config: {}


DEBUG flwr 2023-03-02 18:53:03,408 | server.py:179 | evaluate_round 5 received 3 results and 0 failures
INFO flwr 2023-03-02 18:53:03,410 | server.py:144 | FL finished in 83.45259305596119
INFO flwr 2023-03-02 18:53:03,412 | app.py:202 | app_fit: losses_distributed [(1, 2.3347040017445884), (2, 2.3402809302012124), (3, 2.3355220953623452), (4, 2.3503216902414956), (5, 2.3135551611582437)]
INFO flwr 2023-03-02 18:53:03,414 | app.py:203 | app_fit: metrics_distributed {'accuracy': [(1, 0.10999999940395355), (2, 0.08666666721304257), (3, 0.14000000059604645), (4, 0.1116666632393996), (5, 0.08666666597127914)]}
INFO flwr 2023-03-02 18:53:03,415 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-02 18:53:03,416 | app.py:205 | app_fit: metrics_centralized {}


7/7 [==============================] - 0s 2ms/step - loss: 2.3148 - accuracy: 0.0800
(launch_and_evaluate pid=48116) [Client 3] loss:2.3148365020751953, accuracy:0.07999999821186066


History (loss, distributed):
	round 1: 2.3347040017445884
	round 2: 2.3402809302012124
	round 3: 2.3355220953623452
	round 4: 2.3503216902414956
	round 5: 2.3135551611582437
History (metrics, distributed):
{'accuracy': [(1, 0.10999999940395355), (2, 0.08666666721304257), (3, 0.14000000059604645), (4, 0.1116666632393996), (5, 0.08666666597127914)]}

We will revisit the definition of custom strategies in the following unit to define our own strategy and attempt to minimize some of the challenges that federated learning must address.

### Exercise

Now is your turn, why not you try to run your own architecture with this approach. Beaware of the high requirements when we are in a simulated environment.

In [9]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s

# Code to load the dataset
def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into 10 partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    return train_ds, val_ds, test_ds


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

#########################################################
#    TODO -> Model, Client, client_fn and simulation    #
#########################################################

NUM_CLIENTS = 3

# MobileNetV2
def generate_ann():
    model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer, "sparse_categorical_crossentropy", metrics=["accuracy"])    
    return model

def client_fn(cid) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #Take the appropiate part of the dataset
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    #Create and return the Client
    return FlowerClient(cid, net, trainloader, valloader)

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1,
        fraction_evaluate=1,
        min_fit_clients=NUM_CLIENTS,
        min_evaluate_clients=NUM_CLIENTS,
        min_available_clients=NUM_CLIENTS,
        # Initialization using a random client  |  initial_parameters=fl.common.ndarrays_to_parameters(params),
        evaluate_metrics_aggregation_fn=weighted_average,  # put the metric aggregation for the evaluation
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)

(launch_and_evaluate pid=48116) 


INFO flwr 2023-03-02 18:53:06,351 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-03-02 18:53:16,044	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-02 18:53:17,639 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 4.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 1497125683.0, 'memory': 2994251367.0}
INFO flwr 2023-03-02 18:53:17,640 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-02 18:53:17,641 | server.py:270 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=48162) 2023-03-02 18:53:19.765346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_get_parameters pid=48162) To enable them in other operations, rebuild Tensor

(launch_and_get_parameters pid=48162) [Client 1] get_parameters


(launch_and_fit pid=48160) 2023-03-02 18:53:37.449322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_fit pid=48160) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(launch_and_fit pid=48161) 2023-03-02 18:53:37.439965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_fit pid=48161) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(launch_and_fit pid=48162) [Client 1] fit, config: {}


...

(launch_and_evaluate pid=48162) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=48161) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=48160) [Client 2] evaluate, config: {}
7/7 [==============================] - 3s 38ms/step - loss: 2.3202 - accuracy: 0.0800
(launch_and_evaluate pid=48162) [Client 1] loss:2.3202295303344727, accuracy:0.07999999821186066


DEBUG flwr 2023-03-02 18:57:42,889 | server.py:179 | evaluate_round 5 received 3 results and 0 failures
INFO flwr 2023-03-02 18:57:42,890 | server.py:144 | FL finished in 247.4129567380296
INFO flwr 2023-03-02 18:57:42,901 | app.py:202 | app_fit: losses_distributed [(1, 2.305112838745117), (2, 2.307441314061483), (3, 2.338364521662394), (4, 2.310194730758667), (5, 2.3589529196421304)]
INFO flwr 2023-03-02 18:57:42,903 | app.py:203 | app_fit: metrics_distributed {'accuracy': [(1, 0.09166666865348816), (2, 0.07666666681567828), (3, 0.13000000019868216), (4, 0.1233333299557368), (5, 0.07666666681567828)]}
INFO flwr 2023-03-02 18:57:42,906 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-02 18:57:42,914 | app.py:205 | app_fit: metrics_centralized {}


7/7 [==============================] - 4s 31ms/step - loss: 2.3794 - accuracy: 0.0900
(launch_and_evaluate pid=48161) [Client 0] loss:2.379437208175659, accuracy:0.09000000357627869
3/7 [===========>..................] - ETA: 0s - loss: 2.3807 - accuracy: 0.0833 


History (loss, distributed):
	round 1: 2.305112838745117
	round 2: 2.307441314061483
	round 3: 2.338364521662394
	round 4: 2.310194730758667
	round 5: 2.3589529196421304
History (metrics, distributed):
{'accuracy': [(1, 0.09166666865348816), (2, 0.07666666681567828), (3, 0.13000000019868216), (4, 0.1233333299557368), (5, 0.07666666681567828)]}

7/7 [==============================] - 4s 30ms/step - loss: 2.3772 - accuracy: 0.0600
(launch_and_evaluate pid=48160) [Client 2] loss:2.3771920204162598, accuracy:0.05999999865889549


# Aggregation

To conclude this lesson, let's take a closer look at the key point of these strategies, which is the aggregation algorithm. These algorithms are responsible for combining the updates from the clients to generate the global model, and they are defined in the strategies as we have seen. Generally speaking, there are several types of aggregation that can be used in federated learning (Reddi et. al, 2020).  

Here are the different types of aggregation that can be used in federated learning:

* Federated averaging (`flwr.server.strategy.FedAvg`): In this approach, each device computes an update to the model parameters based on its local data, and these updates are then averaged together to create the global model. This approach is simple and effective, but it can be sensitive to the size of the updates and the quality of the data on each device.

* Federated weighted averaging: This approach is similar to federated averaging, but each device's update is given a different weight based on the size of its data set or the quality of its data. This can help to give more influence to devices with larger or higher-quality data.

* Federated averaging with momentum (`flwr.server.strategy.FedAvgM`): This approach is similar to federated averaging, but it incorporates a momentum term in order to smooth out the updates and help the model converge more quickly.

* Federated stochastic gradient descent(`flwr.server.strategy.FedAdagrad`): In this approach, each device computes an update to the model parameters based on a small batch of its local data, rather than the entire data set. This can help to reduce the communication overhead and improve the convergence rate of the model.

* Federated ADAM (`flwr.server.strategy.FedAdam`): This approach is a variant of federated stochastic gradient descent that uses the ADAM optimization algorithm to adaptively adjust the learning rate based on the gradient and second moment estimates.



All of the previously mentioned aggregation methods, except for Federated Weighted Averaging, are implemented in the `flwr` framework and can be used with the different strategies. Additionally, there are other less common aggregation methods that can be employed. The choice of aggregation method will ultimately depend on the specific characteristics of the data and the requirements of the task at hand.


#### References
* Hard, A., Konečný, J., McMahan, H. B., Richemond-Barakat, C., Sivek, J. S., & Talwar, K. (2018). Federated learning: Strategies for improving communication efficiency. arXiv preprint arXiv:1812.02903.
* Li, Y., Bonawitz, K., & Talwar, K. (2020). Fedprox: An optimizer for communication-efficient federated learning. arXiv preprint arXiv:2002.04283.
* McMahan, H. B., Moore, E., Ramage, D., Hampson, S., & y Arcas, B. A. (2016). Communication-efficient learning of deep networks from decentralized data. arXiv preprint arXiv:1602.05629.
* Yoon, J., Hard, A., Konečný, J., McMahan, H. B., & Sohl-Dickstein, J. (2018). Federal regression: A simple and scalable method for heterogeneous federated learning. arXiv preprint arXiv:1812.03862.
* Reddi, S., Charles, Z., Zaheer, M., Garrett, Z., Rush, K., Konečný, J., Kumar, S. and McMahan, H.B., 2020. Adaptive federated optimization. arXiv preprint arXiv:2003.00295.